In [1]:
import os
from datetime import datetime
from PIL import Image
import numpy as np

# Set the settings module
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "server.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
# Set up Django
django.setup()

from emimage.models import EMImage
from mims.models import MIMSImage, MIMSImageSet
from mims.services import create_alignment_estimates


items= [{
    "roi": 12,
    "x_offset": 23233,
    "y_offset": 36625
}, 
{
    "roi": 13,
    "x_offset": 24050,
    "y_offset": 29630
}, 
{
    "roi": 14,
    "x_offset": 24460,
    "y_offset": 22950
}, 
{
    "roi": 15,
    "x_offset": 25300,
    "y_offset": 15870
}, 
{
    "roi": 16,
    "x_offset": 25300,
    "y_offset": 15870
}]


In [105]:
em = EMImage.objects.get(id="e8dd58a2-036a-4b65-86c0-2d820cba7db8")

roi = 12

config = next((r for r in items if r["roi"] == roi))
x_offset = config["x_offset"]
y_offset = config["y_offset"]

for mims_image in em.mimsimageset_set.first().mims_images.all():
    if f"963_ROI_{roi}.im" not in mims_image.file.name:
        continue
    out_dir = f"/Users/chris/Documents/lab/CD2_mouse_data/roi{roi}"
    rotation = 262
    flip = True
    zoom = 0.0807
    edge_size = 8100

    # Get the MIMS image
    mims = Image.open(os.path.join(out_dir, "32S_autocontrast.png"))
    mims = mims.rotate(angle=360-rotation, expand=True)
    if flip:
        mims = mims.transpose(Image.Transpose.FLIP_LEFT_RIGHT)
    #display(mims)
    #mims = mims.resize((edge_size, edge_size))
    #mims.save(os.path.join(out_dir, "32S_scaled_small.png"))

    
    # Get the SEM crop
    sem = np.asarray(Image.open(em.file.path))
    sem = sem[y_offset:y_offset+edge_size, x_offset:x_offset+edge_size]
    sem = Image.fromarray(sem.astype(np.uint8))
    sem = sem.resize((mims.size[0], mims.size[1]))
    #display(sem)
    #sem.save(os.path.join(out_dir, "sem_small.png"))
    
    print("done")
    

done


In [2]:
for roi in range(12, 16):
    out_dir = f"/Users/chris/Documents/lab/CD2_mouse_data/roi{roi}"
    in_13c = Image.open(os.path.join(out_dir, "13C12C_ratio_WARPED.tiff"))
    in_sem = Image.open(os.path.join(out_dir, "sem.tiff"))
    in_13c = in_13c.resize((in_sem.size[0], in_sem.size[0]))
    in_13c.save(os.path.join(out_dir, "13C12C_ratio_final.tiff"))
    